In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 5.5MB/s 
     |████████████████████████████████| 1.2MB 8.5MB/s 
     |████████████████████████████████| 194kB 11.7MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
Mounted at /content/gdrive


In [2]:
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [6]:
(path/"train").ls()
# what I need to do:
# I need to load in the dataset as tensors, and create a corresponding y tensor with the right labels

(#2) [Path('train/3'),Path('train/7')]

In [34]:
threes = torch.stack([tensor(Image.open(im)) for im in (path/"train"/'3').ls().sorted()])
sevens = torch.stack([tensor(Image.open(im)) for im in (path/"train"/'7').ls().sorted()])
threes.shape, sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [37]:
x_train = torch.cat((threes, sevens)).view(-1, 28*28) / 255.0; x_train.shape
x_train.shape, x_train.type()

torch.Size([12396, 784])

In [32]:
y_train = tensor([0]*len(threes) + [1]*len(sevens)).unsqueeze(1); y_train.shape, y_train.type()

torch.Size([12396, 1])

In [42]:
train_dset = list(zip(x_train, y_train))

In [48]:
# Now do the same for the validation set
threes_valid = torch.stack([tensor(Image.open(im)) for im in (path/"valid"/'3').ls().sorted()])
sevens_valid = torch.stack([tensor(Image.open(im)) for im in (path/"valid"/'7').ls().sorted()])

x_valid = torch.cat((threes_valid, sevens_valid)).view(-1, 28*28) / 255.0; x_train.shape
y_valid = tensor([0]*len(threes_valid) + [1]*len(sevens_valid)).unsqueeze(1); y_valid.shape, y_valid.type()

valid_dset = list(zip(x_valid, y_valid))

In [53]:
train_dl = DataLoader(train_dset, shuffle=True, bs=64)
valid_dl = DataLoader(valid_dset, shuffle=False, bs=64)
dls = DataLoaders(train_dl, valid_dl)

In [59]:
dls.train.one_batch()[0].shape

torch.Size([64, 784])